* count file statistics for Pcap、honepot

In [1]:
import pickle
import numpy as np
import pandas as pd
import os,sys
import random
from tqdm import tqdm
from collections import Counter
import hashlib

In [75]:
root_dir = "../data_hdd4t/"
hp_date_first = True #先分日期才分四種hp?
pcap_date_dir = False #pcap資料夾有多一層日期資料夾?

找當前root_dir中的所有電信商最大範圍日期做為基準
* list

In [76]:
isps = next(os.walk(root_dir))[1]
if 'csv' in isps:
    isps.remove('csv')
if 'meta_data' in isps:
    isps.remove('meta_data')
# pcap date
all_dates = []
for isp in isps:
#     ori_dirs = next(os.walk(root_dir + isp))[1] #honeypot/pcap
    pcap_dir = root_dir + isp + '/pcap/'
    hp_dir = root_dir + isp + '/honeypot/'
    if pcap_date_dir == False:
        pcap_files = next(os.walk(pcap_dir))[2]
        pcap_files = list(filter(lambda f: f.endswith(".tar.gz"), pcap_files))
        dates = [x.split('.')[1] for x in pcap_files]
        dates = [x.replace('-','_') for x in dates]
        all_dates.extend(dates)
    else:
        pcap_dates = next(os.walk(pcap_dir))[1] #csv/meta_data need to be excluded
        all_dates.extend(pcap_dates)
    if hp_date_first:
        hp_dates = next(os.walk(hp_dir))[1]
        dates = [x.replace('-','_') for x in hp_dates]
        all_dates.extend(dates)
    else:
        hp_types_dir = next(os.walk(hp_dir))[1]
        for hp_type in hp_types_dir:
            hp_dates = next(os.walk(hp_dir + hp_type + '/'))[1]
            all_dates.extend(hp_dates)
        
#     for o_d in ori_dirs:
all_dates = sorted(list(set(all_dates)))
all_dates

['2020_01_01',
 '2020_01_02',
 '2020_01_03',
 '2020_01_04',
 '2020_01_05',
 '2020_01_06',
 '2020_01_07',
 '2020_01_08',
 '2020_01_09',
 '2020_01_10',
 '2020_01_11']

In [77]:
save_dir = '../spark_data/data/metadataCount/0101_0111/'
pickle.dump(obj=all_dates,file=open(save_dir+'dates.pkl','wb'))

In [78]:
df = pd.DataFrame(columns=['ISP', 'pcap_loss_date',
                           'amun_loss_date','cowrie_loss_date','dionaea_loss_date','glastopf_loss_date'])
for isp in isps:
    pcap_dir = root_dir + isp + '/pcap/'
    hp_dir = root_dir + isp + '/honeypot/'
    pcap_loss = []
    amun_loss = []
    cowire_loss = []
    dionaea_loss = []
    glastopf_loss = []
    if pcap_date_dir == False:
        pcap_files = list(filter(lambda f: f.endswith(".tar.gz"), pcap_files))
        dates = [x.split('.')[1] for x in pcap_files]
        dates = [x.replace('-','_') for x in dates]
        for a_date in all_dates:
            if a_date not in dates:
                pcap_loss.append(a_date)
    else:
        pcap_dates = next(os.walk(pcap_dir))[1]
        for a_date in all_dates:
            if a_date not in pcap_dates:
                pcap_loss.append(a_date)
        for date in pcap_dates:
            if len(os.listdir(pcap_dir+date) ) == 0:
                pcap_loss.append(date)
    if hp_date_first:
        hp_dates = next(os.walk(hp_dir))[1]
        dates = [x.replace('-','_') for x in hp_dates]
        for a_date in all_dates:
            if a_date not in dates:
#                 print(isp)
                amun_loss.append(a_date)
                cowire_loss.append(a_date)                        
                dionaea_loss.append(a_date)                        
                glastopf_loss.append(a_date)
        for date in hp_dates:
            try:
                date_ = date.replace('-','_')
                if len(os.listdir(hp_dir+date+'/amun') ) == 0:
                    amun_loss.append(date_)
            except FileNotFoundError:
                amun_loss.append(date_)
            try:
                date_ = date.replace('-','_')
                if len(os.listdir(hp_dir+date+'/cowrie') ) == 0:
                    cowire_loss.append(date_)
            except FileNotFoundError:
                cowire_loss.append(date_)
            try:
                date_ = date.replace('-','_')
                if len(os.listdir(hp_dir+date+'/dionaea') ) == 0:
                    dionaea_loss.append(date_)
            except FileNotFoundError:
                dionaea_loss.append(date_)
            try:
                date_ = date.replace('-','_')
                if len(os.listdir(hp_dir+date+'/glastopf') ) == 0:
                    glastopf_loss.append(date_)
            except FileNotFoundError:
                glastopf_loss.append(date_)
    else:
        for a_date in all_dates:
            try:
                if len(os.listdir(hp_dir+'amun/'+a_date) ) == 0:
                    amun_loss.append(a_date)
            except FileNotFoundError:
                amun_loss.append(a_date)
            try:
                if len(os.listdir(hp_dir+'cowrie/'+a_date) ) == 0:
                    cowire_loss.append(a_date)
            except FileNotFoundError:
                cowire_loss.append(a_date)
            try:
                if len(os.listdir(hp_dir+'dionaea/'+a_date) ) == 0:
                    dionaea_loss.append(a_date)
            except FileNotFoundError:
                dionaea_loss.append(a_date)
            try:
                if len(os.listdir(hp_dir+'glastopf/'+a_date) ) == 0:
                    glastopf_loss.append(a_date)
            except FileNotFoundError:
                glastopf_loss.append(a_date)
    temp = [isp, "、".join(pcap_loss), 
            "、".join(amun_loss),"、".join(cowire_loss),"、".join(dionaea_loss),"、".join(glastopf_loss)]
    temp = pd.Series(temp,index=df.columns)
    df = df.append(temp, ignore_index=True)
                
df


,ISP,pcap_loss_date,amun_loss_date,cowrie_loss_date,dionaea_loss_date,glastopf_loss_date
0,亞太,2020_01_01、2020_01_02、2020_01_03、2020_01_04,2020_01_01、2020_01_02、2020_01_03、2020_01_04,2020_01_01、2020_01_02、2020_01_03、2020_01_04,2020_01_01、2020_01_02、2020_01_03、2020_01_04,2020_01_01、2020_01_02、2020_01_03、2020_01_04
1,台灣碩網,2020_01_01、2020_01_02、2020_01_03、2020_01_04,,,,2020_01_02
2,中嘉寬頻,2020_01_01、2020_01_02、2020_01_03、2020_01_04,2020_01_01、2020_01_02、2020_01_03、2020_01_04、20...,2020_01_01、2020_01_02、2020_01_03、2020_01_04、20...,2020_01_01、2020_01_02、2020_01_03、2020_01_04、20...,2020_01_01、2020_01_02、2020_01_03、2020_01_04、20...
3,凱擘,2020_01_01、2020_01_02、2020_01_03、2020_01_04,2020_01_01、2020_01_02、2020_01_03、2020_01_04,2020_01_01、2020_01_02、2020_01_03、2020_01_04,2020_01_01、2020_01_02、2020_01_03、2020_01_04,2020_01_01、2020_01_02、2020_01_03、2020_01_04
4,台灣之星,2020_01_01、2020_01_02、2020_01_03、2020_01_04,2020_01_01、2020_01_02、2020_01_03、2020_01_04、20...,2020_01_01、2020_01_02、2020_01_03、2020_01_04、20...,2020_01_01、2020_01_02、2020_01_03、2020_01_04、20...,2020_01_01、2020_01_02、2020_01_03、2020_01_04、20...
5,遠傳,2020_01_01、2020_01_02、2020_01_03、2020_01_04,2020_01_01、2020_01_02、2020_01_03、2020_01_04、20...,2020_01_01、2020_01_02、2020_01_03、2020_01_04,2020_01_01、2020_01_02、2020_01_03、2020_01_04、20...,2020_01_01、2020_01_02、2020_01_03、2020_01_04、20...
6,台哥大,2020_01_01、2020_01_02、2020_01_03、2020_01_04,2020_01_01、2020_01_02、2020_01_03、2020_01_04,2020_01_01、2020_01_02、2020_01_03、2020_01_04,2020_01_01、2020_01_02、2020_01_03、2020_01_04,2020_01_01、2020_01_02、2020_01_03、2020_01_04
7,台基開發,2020_01_01、2020_01_02、2020_01_03、2020_01_04,2020_01_01、2020_01_02、2020_01_03、2020_01_04,2020_01_01、2020_01_02、2020_01_03、2020_01_04、20...,2020_01_01、2020_01_02、2020_01_03、2020_01_04,2020_01_01、2020_01_02、2020_01_03、2020_01_04
8,中華電信,2020_01_01、2020_01_02、2020_01_03、2020_01_04,2020_01_02、2020_01_03、2020_01_04、2020_01_05、20...,2020_01_02、2020_01_03、2020_01_04、2020_01_05、20...,2020_01_02、2020_01_03、2020_01_04、2020_01_05、20...,2020_01_02、2020_01_03、2020_01_04、2020_01_05、20...
9,台灣固網,2020_01_01、2020_01_02、2020_01_03、2020_01_04,2020_01_01、2020_01_02、2020_01_03、2020_01_04,2020_01_01、2020_01_02、2020_01_03、2020_01_04,2020_01_01、2020_01_02、2020_01_03、2020_01_04,2020_01_01、2020_01_02、2020_01_03、2020_01_04


In [79]:
pickle.dump(obj=df,file=open(save_dir+'lost_dates.pkl','wb'))

True

In [98]:
pkl_dir = '../spark_data/data/metadataCount/'
dates_dir = next(os.walk(pkl_dir))[1]
dates_dir

['1229_0112', '0112_0125', '0101_0111', '0126_0208']

In [99]:
# check date range
all_dates_li = []
for dates in dates_dir:
    date_li = pickle.load(open(pkl_dir+dates+'/dates.pkl','rb'))
    all_dates_li.extend(date_li)
all_dates_li

['2019_12_29',
 '2019_12_30',
 '2019_12_31',
 '2020_01_01',
 '2020_01_02',
 '2020_01_03',
 '2020_01_04',
 '2020_01_11',
 '2020_01_12',
 '2020_01_12',
 '2020_01_13',
 '2020_01_14',
 '2020_01_15',
 '2020_01_16',
 '2020_01_17',
 '2020_01_18',
 '2020_01_19',
 '2020_01_20',
 '2020_01_21',
 '2020_01_22',
 '2020_01_23',
 '2020_01_24',
 '2020_01_25',
 '2020_01_01',
 '2020_01_02',
 '2020_01_03',
 '2020_01_04',
 '2020_01_05',
 '2020_01_06',
 '2020_01_07',
 '2020_01_08',
 '2020_01_09',
 '2020_01_10',
 '2020_01_11',
 '2020_01_26',
 '2020_01_27',
 '2020_01_28',
 '2020_01_29',
 '2020_01_30',
 '2020_01_31',
 '2020_02_01',
 '2020_02_02',
 '2020_02_03',
 '2020_02_04',
 '2020_02_05',
 '2020_02_06',
 '2020_02_07',
 '2020_02_08']

In [100]:
count_dict = dict(Counter(all_dates_li))
count_dict = {k: v for k, v in sorted(count_dict.items(), key=lambda item: item[1],reverse=True)}
count_dict

{'2020_01_01': 2,
 '2020_01_02': 2,
 '2020_01_03': 2,
 '2020_01_04': 2,
 '2020_01_11': 2,
 '2020_01_12': 2,
 '2019_12_29': 1,
 '2019_12_30': 1,
 '2019_12_31': 1,
 '2020_01_13': 1,
 '2020_01_14': 1,
 '2020_01_15': 1,
 '2020_01_16': 1,
 '2020_01_17': 1,
 '2020_01_18': 1,
 '2020_01_19': 1,
 '2020_01_20': 1,
 '2020_01_21': 1,
 '2020_01_22': 1,
 '2020_01_23': 1,
 '2020_01_24': 1,
 '2020_01_25': 1,
 '2020_01_05': 1,
 '2020_01_06': 1,
 '2020_01_07': 1,
 '2020_01_08': 1,
 '2020_01_09': 1,
 '2020_01_10': 1,
 '2020_01_26': 1,
 '2020_01_27': 1,
 '2020_01_28': 1,
 '2020_01_29': 1,
 '2020_01_30': 1,
 '2020_01_31': 1,
 '2020_02_01': 1,
 '2020_02_02': 1,
 '2020_02_03': 1,
 '2020_02_04': 1,
 '2020_02_05': 1,
 '2020_02_06': 1,
 '2020_02_07': 1,
 '2020_02_08': 1}

In [101]:
def aliase_convert(s):
    '''
    dictionary baesd conversion
    '''
    if s == '台哥大':
        return '台灣大哥大'
    elif s == '台基開發':
        return '台灣基礎開發'
    elif s == '亞太':
        return '亞太電信'
    elif s == '遠傳':
        return '遠傳電信'
    else:
        return s

In [102]:
# load df
all_df = None
for i,dates in enumerate(dates_dir):
    df = pickle.load(open(pkl_dir+dates+'/lost_dates.pkl','rb'))
    df['ISP'] = df.ISP.map(aliase_convert)
    df = df.sort_values('ISP')
    df = df.reset_index(drop=True)
    if i == 0:
        all_df = df
    else:
        all_df = all_df + "、" + df
all_df

,ISP,pcap_loss_date,amun_loss_date,cowrie_loss_date,dionaea_loss_date,glastopf_loss_date
0,中嘉寬頻、中嘉寬頻、中嘉寬頻、中嘉寬頻,2020_01_11、2020_01_12、、2020_01_01、2020_01_02、2...,2019_12_29、2020_01_02、2020_01_03、2020_01_04、20...,2019_12_29、2020_01_02、2020_01_03、2020_01_04、20...,2019_12_29、2020_01_02、2020_01_03、2020_01_04、20...,2019_12_29、2020_01_02、2020_01_03、2020_01_04、20...
1,中華電信、中華電信、中華電信、中華電信,2020_01_11、、2020_01_01、2020_01_02、2020_01_03、2...,2019_12_29、2020_01_02、2020_01_03、2020_01_04、20...,2019_12_29、2020_01_02、2020_01_03、2020_01_04、20...,2019_12_29、2020_01_02、2020_01_03、2020_01_04、20...,2019_12_29、2020_01_02、2020_01_03、2020_01_04、20...
2,亞太電信、亞太電信、亞太電信、亞太電信,2019_12_31、2020_01_11、、2020_01_01、2020_01_02、2...,2019_12_31、2020_01_01、2020_01_11、2020_01_25、20...,2019_12_31、2020_01_01、2020_01_11、2020_01_25、20...,2019_12_31、2020_01_01、2020_01_11、2020_01_25、20...,2019_12_31、2020_01_01、2020_01_11、2020_01_25、20...
3,凱擘、凱擘、凱擘、凱擘,2019_12_31、2020_01_11、2020_01_12、、2020_01_01、2...,2019_12_31、2020_01_01、2020_01_11、2020_01_12、、2...,2019_12_31、2020_01_01、2020_01_11、2020_01_12、、2...,2019_12_30、2019_12_31、2020_01_01、2020_01_11、20...,2019_12_31、2020_01_01、2020_01_11、2020_01_12、、2...
4,台固媒體、台固媒體、台固媒體、台固媒體,2019_12_31、2020_01_11、2020_01_12、、2020_01_01、2...,2020_01_11、2020_01_12、、2020_01_01、2020_01_02、2...,2020_01_02、2020_01_11、2020_01_12、2020_01_17、20...,2019_12_29、2019_12_30、2020_01_11、2020_01_12、、2...,2020_01_11、2020_01_12、、2020_01_01、2020_01_02、2...
5,台灣之星、台灣之星、台灣之星、台灣之星,2019_12_30、2019_12_31、2020_01_12、、2020_01_01、2...,2020_01_11、2020_01_12、、2020_01_01、2020_01_02、2...,2020_01_11、2020_01_12、、2020_01_01、2020_01_02、2...,2020_01_11、2020_01_12、、2020_01_01、2020_01_02、2...,2020_01_11、2020_01_12、、2020_01_01、2020_01_02、2...
6,台灣固網、台灣固網、台灣固網、台灣固網,2020_01_11、2020_01_12、、2020_01_01、2020_01_02、2...,2020_01_11、2020_01_12、、2020_01_01、2020_01_02、2...,2020_01_11、2020_01_12、、2020_01_01、2020_01_02、2...,2020_01_11、2020_01_12、、2020_01_01、2020_01_02、2...,2020_01_11、2020_01_12、、2020_01_01、2020_01_02、2...
7,台灣基礎開發、台灣基礎開發、台灣基礎開發、台灣基礎開發,2019_12_30、2019_12_31、2020_01_11、2020_01_12、、2...,2020_01_11、2020_01_12、、2020_01_01、2020_01_02、2...,2019_12_30、2019_12_31、2020_01_11、2020_01_12、20...,2020_01_11、2020_01_12、、2020_01_01、2020_01_02、2...,2020_01_11、2020_01_12、、2020_01_01、2020_01_02、2...
8,台灣大哥大、台灣大哥大、台灣大哥大、台灣大哥大,2019_12_29、2019_12_30、2019_12_31、2020_01_11、、2...,2020_01_11、、2020_01_01、2020_01_02、2020_01_03、2...,2020_01_11、、2020_01_01、2020_01_02、2020_01_03、2...,2019_12_31、2020_01_01、2020_01_11、、2020_01_01、2...,2020_01_11、、2020_01_01、2020_01_02、2020_01_03、2...
9,台灣碩網、台灣碩網、台灣碩網、台灣碩網,2019_12_31、2020_01_11、、2020_01_01、2020_01_02、2...,2019_12_31、2020_01_01、2020_01_11、2020_01_13、20...,2019_12_31、2020_01_01、2020_01_11、2020_01_25、、2...,2019_12_31、2020_01_01、2020_01_11、2020_01_25、、2...,2019_12_31、2020_01_01、2020_01_02、2020_01_11、20...


In [104]:
def process_ISP(s):
    return s.split("、")[0]
def process_Date(s):
    dates = s.split("、")
    temp = list(filter(None, dates))
    temp = dict(Counter(temp))
    dict_keys = sorted(list(temp.keys()))
    lost_dates = []
    for key in dict_keys:
        if key in lost_dates:
            continue
        if temp[key] >= count_dict[key]:
            lost_dates.append(key)
    return "、".join(lost_dates)    

In [105]:
all_df['ISP'] = all_df.ISP.map(process_ISP)
all_df['pcap_loss_date'] = all_df.pcap_loss_date.map(process_Date)
all_df['amun_loss_date'] = all_df.amun_loss_date.map(process_Date)
all_df['cowrie_loss_date'] = all_df.cowrie_loss_date.map(process_Date)
all_df['dionaea_loss_date'] = all_df.dionaea_loss_date.map(process_Date)
all_df['glastopf_loss_date'] = all_df.glastopf_loss_date.map(process_Date)
sort_days = sorted(list(count_dict.keys()))
print(sort_days)
all_df

['2019_12_29', '2019_12_30', '2019_12_31', '2020_01_01', '2020_01_02', '2020_01_03', '2020_01_04', '2020_01_05', '2020_01_06', '2020_01_07', '2020_01_08', '2020_01_09', '2020_01_10', '2020_01_11', '2020_01_12', '2020_01_13', '2020_01_14', '2020_01_15', '2020_01_16', '2020_01_17', '2020_01_18', '2020_01_19', '2020_01_20', '2020_01_21', '2020_01_22', '2020_01_23', '2020_01_24', '2020_01_25', '2020_01_26', '2020_01_27', '2020_01_28', '2020_01_29', '2020_01_30', '2020_01_31', '2020_02_01', '2020_02_02', '2020_02_03', '2020_02_04', '2020_02_05', '2020_02_06', '2020_02_07', '2020_02_08']


,ISP,pcap_loss_date,amun_loss_date,cowrie_loss_date,dionaea_loss_date,glastopf_loss_date
0,中嘉寬頻,,2019_12_29、2020_01_02、2020_01_03、2020_01_04、20...,2019_12_29、2020_01_02、2020_01_03、2020_01_04、20...,2019_12_29、2020_01_02、2020_01_03、2020_01_04、20...,2019_12_29、2020_01_02、2020_01_03、2020_01_04、20...
1,中華電信,,2019_12_29、2020_01_02、2020_01_03、2020_01_04、20...,2019_12_29、2020_01_02、2020_01_03、2020_01_04、20...,2019_12_29、2020_01_02、2020_01_03、2020_01_04、20...,2019_12_29、2020_01_02、2020_01_03、2020_01_04、20...
2,亞太電信,2019_12_31,2019_12_31、2020_01_01、2020_01_25、2020_01_27、20...,2019_12_31、2020_01_01、2020_01_25、2020_01_27、20...,2019_12_31、2020_01_01、2020_01_25、2020_01_27、20...,2019_12_31、2020_01_01、2020_01_25、2020_01_27、20...
3,凱擘,2019_12_31,2019_12_31、2020_01_01,2019_12_31、2020_01_01,2019_12_30、2019_12_31、2020_01_01,2019_12_31、2020_01_01
4,台固媒體,2019_12_31,,2020_01_02、2020_01_17,2019_12_29、2019_12_30,
5,台灣之星,2019_12_30、2019_12_31,2020_01_11,2020_01_11,2020_01_11,2020_01_11
6,台灣固網,,,,,
7,台灣基礎開發,2019_12_30、2019_12_31,,2019_12_30、2019_12_31、2020_01_05、2020_01_06、20...,,
8,台灣大哥大,2019_12_29、2019_12_30、2019_12_31,,,2019_12_31、2020_01_01,
9,台灣碩網,2019_12_31,2019_12_31、2020_01_13、2020_01_25、2020_01_27、20...,2019_12_31、2020_01_25、2020_01_27、2020_01_28、20...,2019_12_31、2020_01_25、2020_01_27、2020_01_28、20...,2019_12_31、2020_01_02、2020_01_25、2020_01_27、20...


In [107]:
# sort_days = sorted(list(count_dict.keys()))
all_df.to_excel(pkl_dir+sort_days[0]+'-'+sort_days[-1]+'_lostDates_rev.xlsx')

In [85]:
temp = all_df.iloc[10,2]
temp = temp.split("、")
temp = list(filter(None, temp))
temp = dict(Counter(temp))
temp

{'2020_01_11': 1,
 '2020_01_01': 1,
 '2020_01_02': 1,
 '2020_01_03': 1,
 '2020_01_04': 1,
 '2020_01_06': 1}

In [70]:
temp['2020_01_04']

1

In [87]:
dict_keys = list(temp.keys())
lost_dates = []
for key in dict_keys:
    if temp[key] >= count_dict[key]:
        lost_dates.append(key)
"、".join(lost_dates)

'2020_01_06'

In [68]:
count_dict

{'2020_01_01': 2,
 '2020_01_02': 2,
 '2020_01_03': 2,
 '2020_01_04': 2,
 '2020_01_11': 2,
 '2020_01_12': 2,
 '2019_12_29': 1,
 '2019_12_30': 1,
 '2019_12_31': 1,
 '2020_01_13': 1,
 '2020_01_14': 1,
 '2020_01_15': 1,
 '2020_01_16': 1,
 '2020_01_17': 1,
 '2020_01_18': 1,
 '2020_01_19': 1,
 '2020_01_20': 1,
 '2020_01_21': 1,
 '2020_01_22': 1,
 '2020_01_23': 1,
 '2020_01_24': 1,
 '2020_01_25': 1,
 '2020_01_05': 1,
 '2020_01_06': 1,
 '2020_01_07': 1,
 '2020_01_08': 1,
 '2020_01_09': 1,
 '2020_01_10': 1,
 '2020_01_26': 1,
 '2020_01_27': 1,
 '2020_01_28': 1,
 '2020_01_29': 1,
 '2020_01_30': 1,
 '2020_01_31': 1,
 '2020_02_01': 1,
 '2020_02_02': 1,
 '2020_02_03': 1,
 '2020_02_04': 1,
 '2020_02_05': 1,
 '2020_02_06': 1,
 '2020_02_07': 1,
 '2020_02_08': 1}